In [1]:
import importlib
import types
from eth2spec.config.config_util import prepare_config
from eth2spec.utils.ssz.ssz_impl import hash_tree_root

In [2]:
import os, sys
sys.path.insert(1, os.path.realpath(os.path.pardir) + "/notebooks/thunderdome")
import beaconrunner as br

In [3]:
import pandas as pd

In [4]:
prepare_config(".", "fast.yaml") # 4 slots per epoch
br.reload_package(br)

In [5]:
from beaconrunner.specs import SLOTS_PER_EPOCH, SECONDS_PER_SLOT
print(SLOTS_PER_EPOCH)

4



We create our _observers_, to allow us to record interesting metrics at each simulation step.

In [6]:
from beaconrunner.specs import get_beacon_proposer_index

def current_slot(params, step, sL, s, _input):
    return ("current_slot", s["network"].validators[0].data.slot)

def get_current_epoch_proposers(params, step, sL, s, _input):
    network = s["network"]
    validators = s["network"].validators
    validator = validators[0] # We are just assuming validator 0's view is representative
    
    current_epoch = br.specs.get_current_epoch(current_state)
    start_slot = br.specs.compute_start_slot_at_epoch(current_epoch)
    start_state = current_state.copy() if start_slot == current_state.slot else \ 
    validator.store.block_states[br.specs.get_block_root(current_state, current_epoch)].copy()

    if validator.data.slot % SLOTS_PER_EPOCH == 0:
        current_epoch_proposers = []
        for slot in range(start_slot, start_slot * SLOTS_PER_EPOCH):
            if slot < start_state.slot:
                current_proposer_duties += [False]
                continue

            if start_state.slot < slot:
                process_slots(start_state, slot)
            
            current_proposer_duties.append((slot, br.specs.get_beacon_proposer_index(start_state))
            return current_epoch_proposers
    else:
        return None

def get_block_proposer

observers = {
    "current_slot": current_slot,
    "current_epoch_proposers": get_current_proposer_index,
}

In [7]:
import beaconrunner.simulator as simulator
import beaconrunner.network as network
from beaconrunner.validators.ASAPValidator import ASAPValidator

num_validators = 4
validators = [ASAPValidator(i) for i in range(num_validators)]

# Create a genesis state
genesis_state = br.simulator.get_genesis_state(validators, seed="let's play randao")
# Validators load the state
[v.load_state(genesis_state.copy()) for v in validators]

simulator.skip_genesis_block(validators) # nothing to propose at genesis, skip to slot 1!

For simplicity let's just assume a fully connected network, meaning that all validators are connected with each other.

In [8]:
set_a = network.NetworkSet(validators=list([0,1,2,3]))
net = network.Network(validators = validators, sets = list([set_a]))

In [9]:
print("Genesis time =", validators[0].store.genesis_time, "seconds")
print("Store time =", validators[0].store.time, "seconds")
print("Current slot =", validators[0].data.slot)

Genesis time = 1578182400 seconds
Store time = 1578182412 seconds
Current slot = 1


In [10]:
attestation_views = [
    (validator_index, validator.data.current_attest_slot) for validator_index, validator in enumerate(net.validators)
    ]
attestation_views

[(0, 2), (1, 3), (2, 1), (3, 0)]

In [11]:
proposer_views = [(validator_index, validator.data.current_proposer_duties) \
                  for validator_index, validator in enumerate(net.validators)] 
proposer_views

[(0, [False, True, False, False]),
 (1, [False, False, False, True]),
 (2, [False, False, True, False]),
 (3, [True, False, False, False])]

We see that validator 0 is supposed to propose a block in slot 1. So let's go ahead and do that.

In [12]:
block = net.validators[0].propose({ "attestations": [], "blocks": [] })

0 proposing block for slot 1


In [13]:
network.disseminate_block(net,3,block)

Remember that we are assuming a fully connected network, meaning that disseminating the block once should have all validators know about the block. Let's double-check to be sure:

In [14]:
received_block = [(i, validators[i].data.received_block) for i in range(4)]
received_block

[(0, True), (1, True), (2, True), (3, True)]

In [15]:
block_root = hash_tree_root(block.message) # block.message is of type BeaconBlock
net.validators[0].store.blocks[block_root]

BeaconBlock(Container)
    slot: Slot = 1
    proposer_index: ValidatorIndex = 0
    parent_root: Root = 0xad80f24850908baecc5d6b961370c4f6b1a6b0df8093b4bc0843bf5c67beaa6e
    state_root: Root = 0x08fd4a56bb2e456203a35f5477919a8c7dbb44b32a648543e7acc3b9ece318a0
    body: BeaconBlockBody = BeaconBlockBody(Container)
                                randao_reveal: BLSSignature = 0xa6d258974ed36901cde91a15bd73a6f7590a8b0111d42c92cd2cf48bf93ac41115e8ee46508a36b1aa398e6506d5fa0f0596a03a7e2c7b3bec62f037707b7f3eba986c352b0014284a1ef2569c89f823c9cb18a21d6a27cfd3e47162a3149869
                                eth1_data: Eth1Data = Eth1Data(Container)
                                                          deposit_root: Root = 0x0000000000000000000000000000000000000000000000000000000000000000
                                                          deposit_count: uint64 = 0
                                                          block_hash: SpecialByteVectorView = 0x00000000000000000000000000

Let's fast forward in time to the beginning of the next slot

In [16]:
for validator in net.validators:
    validator.forward_by(SECONDS_PER_SLOT)
print("Validator 0 says this is slot number {}".format(net.validators[0].data.slot))

Validator 0 says this is slot number 2


In [17]:
committee_slots = [validator.data.current_attest_slot for validator in net.validators]
pd.DataFrame({ "validator_index": [0, 1, 2, 3], "committee_slot": committee_slots})

,validator_index,committee_slot
0,0,2
1,1,3
2,2,1
3,3,0


According to schedule, at slot 2, validator 0 is expected to attest.

In [18]:
# Validators only attest 4 seconds into a slot, so let's forward time... 
for validator in net.validators:
    validator.forward_by(4)

known_items = network.knowledge_set(net, 2)
attestation = net.validators[0].attest(known_items)

In [19]:
print(attestation)

Attestation(Container)
    aggregation_bits: SpecialBitlistView = Bitlist[64](1 bits: 1)
    data: AttestationData = AttestationData(Container)
                                slot: Slot = 2
                                index: CommitteeIndex = 0
                                beacon_block_root: Root = 0x8eba4bd57ef3e9e641ec742afa43db26ad4cd6f49798888f4d6cbd322927c76b
                                source: Checkpoint = Checkpoint(Container)
                                                         epoch: Epoch = 0
                                                         root: Root = 0x0000000000000000000000000000000000000000000000000000000000000000
                                target: Checkpoint = Checkpoint(Container)
                                                         epoch: Epoch = 0
                                                         root: Root = 0xad80f24850908baecc5d6b961370c4f6b1a6b0df8093b4bc0843bf5c67beaa6e
    signature: BLSSignature = 0xb81e6b2d3169f0dbad0215

In [20]:
import importlib
import types
from eth2spec.config.config_util import prepare_config
from eth2spec.utils.ssz.ssz_impl import hash_tree_root

In [21]:
import os, sys
sys.path.insert(1, os.path.realpath(os.path.pardir) + "/notebooks/thunderdome")

import beaconrunner as br

In [22]:
prepare_config(".", "fast.yaml")
br.reload_package(br)

In [23]:
def current_slot(params, step, sL, s, _input):
    return ("current_slot", s["network"].validators[0].data.slot)

def average_balance_prudent(params, step, sL, s, _input):
    validators = s["network"].validators
    validator = validators[0]
    head = br.specs.get_head(validator.store)
    current_state = validator.store.block_states[head]
    current_epoch = br.specs.get_current_epoch(current_state)
    prudent_indices = [i for i, v in enumerate(validators) if v.validator_behaviour == "prudent"]
    prudent_balances = [b for i, b in enumerate(current_state.balances) if i in prudent_indices]
    return ("average_balance_prudent", br.utils.eth2.gwei_to_eth(sum(prudent_balances) / float(len(prudent_indices))))

def average_balance_asap(params, step, sL, s, _input):
    validators = s["network"].validators
    validator = validators[0]
    head = br.specs.get_head(validator.store)
    current_state = validator.store.block_states[head]
    current_epoch = br.specs.get_current_epoch(current_state)
    asap_indices = [i for i, v in enumerate(validators) if v.validator_behaviour == "asap"]
    asap_balances = [b for i, b in enumerate(current_state.balances) if i in asap_indices]
    return ("average_balance_asap", br.utils.eth2.gwei_to_eth(sum(asap_balances) / float(len(asap_indices))))

observers = {
    "current_slot": current_slot,
    "average_balance_prudent": average_balance_prudent,
    "average_balance_asap": average_balance_asap,
}

In [24]:
from random import sample
from beaconrunner.validators.ASAPValidator import ASAPValidator
from beaconrunner.validators.PrudentValidator import PrudentValidator

def simulate_once(network_sets, num_run, num_validators, network_update_rate):
    # Half our validators are prudent, the others are ASAPs
    num_prudent = int(num_validators / 2)

    # We sample the position on the p2p network of prudent validators randomly
    prudentset = set(sample(range(num_validators), num_prudent))

    validators = []

    # Initiate validators
    for i in range(num_validators):
        if i in prudentset:
            new_validator = PrudentValidator(i)
        else:
            new_validator = ASAPValidator(i)
        validators.append(new_validator)
    
    # Create a genesis state
    genesis_state = br.simulator.get_genesis_state(validators)
    
    # Validators load the state
    [v.load_state(genesis_state.copy()) for v in validators]

    br.simulator.skip_genesis_block(validators) # forward time by SECONDS_PER_SLOT

    network = br.network.Network(validators = validators, sets=network_sets)

    parameters = br.simulator.SimulationParameters({
        "num_epochs": 2,
        "num_run": num_run,
        "frequency": 1,
        "network_update_rate": network_update_rate,
    })

    return br.simulator.simulate(network, parameters, observers)

In [25]:
import pandas as pd

num_validators = 4

# Create the network peers
set_a = br.network.NetworkSet(validators=list(range(0, int(num_validators * 2 / 3.0))))
set_b = br.network.NetworkSet(validators=list(range(int(num_validators / 2.0), num_validators)))
network_sets = list([set_a, set_b])

num_runs = 1
network_update_rate = 0.25

df = pd.concat([simulate_once(network_sets, num_run, num_validators, network_update_rate) for num_run in range(num_runs)])

will simulate 2 epochs ( 8 slots ) at frequency 1 moves/second
total 96 simulation steps

                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 1
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (96, 2, 1, 4)
Initializing configurations: 100%|██████████| 1/1 [00:00<00:00, 140.66it/s]Execution Method: local_simulations
SimIDs   : [0]
SubsetIDs: [0]
Ns       : [0]
ExpIDs   : [0]
Execution Mode: single_threaded
2 proposing block for slot 1
1 proposing block for slot 2
1 proposing block for slot 3

Epoch is being processed...
Epoch is being processed...
2 proposing block for slot 4
Epoch is being processed...
3 proposing block for slot 5
2 proposing block for slot 6
3 proposing block for slot 7
Epoch is being processed...
Epoch is being processed...
Epoch is being processed...

### Experiment A
In slot 30 of epoch 0, forward the state to slot 31 (assuming no empty slot), then to slot 0 of epoch 1, record the proposer

### Experiment B
In slot 31 of epoch 0, a block was proposed for slot 31, then forward the state to slot 0 of epoch 1, record the proposer.

Are the proposers different between experiment A and B?

### Experiment C

The block at slot 31 of epoch 0 has a slashing event. Forward the state to slot 0 of epoch 1. Is it still the same proposer?

Can we confirm the intuition that the proposer in experiment C is different than in experiment A&B?